# Проект A\B-test #

# Содержание #

<a id='purpose'></a>
# Цель проекта #
1. оценка корректности проведения A\B-теста

2. анализ результатов A\B-теста

<a id='specification'></a>
# Техническое задание #
Название теста: recommender_system_test;

Группы:
    * А (контрольная)
    * B (новая платёжная воронка);
Дата запуска: 2020-12-07;

Дата остановки набора новых пользователей: 2020-12-21;

Дата остановки: 2021-01-04;

Аудитория: 15% новых пользователей из региона EU;

Назначение теста: тестирование изменений, связанных с внедрением улучшенной рекомендательной системы;

Ожидаемое количество участников теста: 6000.

Ожидаемый эффект: за 14 дней с момента регистрации в системе пользователи покажут улучшение каждой метрики не менее, чем на 10%:
    * конверсии в просмотр карточек товаров — событие product_page
    * просмотры корзины — product_cart
    * покупки — purchase.

<a id='data_descriptions'></>
# Описание данных #

<b>/datasets/ab_project_marketing_events.csv — календарь маркетинговых событий на 2020 год</b>
    * name — название маркетингового события;
    * regions — регионы, в которых будет проводиться рекламная кампания;
    * start_dt — дата начала кампании;
    * finish_dt — дата завершения кампании.

<b>/datasets/final_ab_new_users.csv — все пользователи, зарегистрировавшиеся в интернет-магазине в период с 7 по 21 декабря 2020 года</b>
    * user_id — идентификатор пользователя;
    * first_date — дата регистрации;
    * region — регион пользователя;
    * device — устройство, с которого происходила регистрация.

<b>/datasets/final_ab_events.csv — все события новых пользователей в период с 7 декабря 2020 по 4 января 2021 года</b>
    * user_id — идентификатор пользователя;
    * event_dt — дата и время события;
    * event_name — тип события;
    * details — дополнительные данные о событии. Например, для покупок, purchase, в этом поле хранится стоимость покупки в долларах.

<b>/datasets/final_ab_participants.csv — таблица участников тестов</b>
    * user_id — идентификатор пользователя;
    * ab_test — название теста;
    * group — группа пользователя.

<a id='setup'></a>
# Установка библиотек #

In [1]:
# !pip install --user plotly==5.8.0
# !pip install --user plotly_express
## !pip install --user pandas-profiling

<a id='import'></a>
# Импорт библиотек #

In [2]:
import os
import warnings
import  pandas as pd
from IPython.core.display_functions import display

pd.set_option('display.max_colwidth', None)
warnings.filterwarnings('ignore')

<a id='load'></a>
# Загрузка данных #

<a id='load_func'></a>
## Функция проверки путей к файлам данных ##

In [3]:
def get_path(filename):
    path_local = 'datasets/'
    path_server = '/datasets/'

    if os.path.exists(path=path_local): return path_local + filename
    elif os.path.exists(path=path_server): return path_server + filename
    else: print('Данные отсутствуют. Проверьте путь к папкам с данными')

In [4]:
marketing_events = pd.read_csv(filepath_or_buffer=get_path(filename='ab_project_marketing_events.csv'))
ab_events = pd.read_csv(filepath_or_buffer=get_path(filename='final_ab_events.csv'))
ab_new_user = pd.read_csv(filepath_or_buffer=get_path(filename='final_ab_new_users.csv'))
ab_participants = pd.read_csv(filepath_or_buffer=get_path(filename='final_ab_participants.csv'))

<a id='load_preview'></a>
## Первичный осмотр данных ##

In [5]:
display(marketing_events.head())
print(marketing_events.info())

,name,regions,start_dt,finish_dt
0,Christmas&New Year Promo,"EU, N.America",2020-12-25,2021-01-03
1,St. Valentine's Day Giveaway,"EU, CIS, APAC, N.America",2020-02-14,2020-02-16
2,St. Patric's Day Promo,"EU, N.America",2020-03-17,2020-03-19
3,Easter Promo,"EU, CIS, APAC, N.America",2020-04-12,2020-04-19
4,4th of July Promo,N.America,2020-07-04,2020-07-11


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   name       14 non-null     object
 1   regions    14 non-null     object
 2   start_dt   14 non-null     object
 3   finish_dt  14 non-null     object
dtypes: object(4)
memory usage: 576.0+ bytes
None


In [6]:
display(ab_events.head())
print(ab_events.info())

,user_id,event_dt,event_name,details
0,E1BDDCE0DAFA2679,2020-12-07 20:22:03,purchase,99.99
1,7B6452F081F49504,2020-12-07 09:22:53,purchase,9.99
2,9CD9F34546DF254C,2020-12-07 12:59:29,purchase,4.99
3,96F27A054B191457,2020-12-07 04:02:40,purchase,4.99
4,1FD7660FDF94CA1F,2020-12-07 10:15:09,purchase,4.99


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   user_id     440317 non-null  object 
 1   event_dt    440317 non-null  object 
 2   event_name  440317 non-null  object 
 3   details     62740 non-null   float64
dtypes: float64(1), object(3)
memory usage: 13.4+ MB
None


In [7]:
ab_events.details.unique()

array([ 99.99,   9.99,   4.99, 499.99,    nan])

In [8]:
display(ab_new_user.head())
print(ab_new_user.info())

,user_id,first_date,region,device
0,D72A72121175D8BE,2020-12-07,EU,PC
1,F1C668619DFE6E65,2020-12-07,N.America,Android
2,2E1BF1D4C37EA01F,2020-12-07,EU,PC
3,50734A22C0C63768,2020-12-07,EU,iPhone
4,E1BDDCE0DAFA2679,2020-12-07,N.America,iPhone


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0 to 61732
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   user_id     61733 non-null  object
 1   first_date  61733 non-null  object
 2   region      61733 non-null  object
 3   device      61733 non-null  object
dtypes: object(4)
memory usage: 1.9+ MB
None


In [9]:
display(ab_participants.head())
print(ab_participants.info())

,user_id,group,ab_test
0,D1ABA3E2887B6A73,A,recommender_system_test
1,A7A3664BD6242119,A,recommender_system_test
2,DABC14FDDFADD29E,A,recommender_system_test
3,04988C5DF189632E,A,recommender_system_test
4,482F14783456D21B,B,recommender_system_test


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18268 entries, 0 to 18267
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  18268 non-null  object
 1   group    18268 non-null  object
 2   ab_test  18268 non-null  object
dtypes: object(3)
memory usage: 428.3+ KB
None


## Итоги предварительного осмотра данных ##
1. marketing_events:
    * start_dt - привести к типу datetime;
    * finish_dt - привести к типу datetime;
    * name - перевести значения в нижний регистр;
    * region - перевести значения в нижний регистр;
    * пропуски данных отсутсвуют;

2. ab_events:
    * user_id - перевести значения в нижний регистр;
    * event_dt - привести к типу datetime;
    * event_name - перевести значение в нижний регистр;
    * details - заменить пропуски на 0;
      Причина: из ТЗ - "дополнительные данные о событии". Заполняется опционально.

3. ab_new_user:
    * user_id - перевести значения в нижний регистр;
    * first_date - привести к типу datetime;
    * region - перевести значения в нижний регистр;
    * device - перевести значения в нижний регистр;
    * пропуски данных отсутсвуют;

4. ab_participants:
    * user_id - перевести значения в нижний регистр;
    * group_id - перевести значения в нижний регистр;
    * ab_test - перевести значения в нижний регистр;

<a id='prepare'></a>
# Предобработка данных #

<a id='prepare_types'></a>
## Приведение типов ##

In [10]:
marketing_events['start_dt'] = marketing_events['start_dt'].astype('datetime64')
marketing_events['finish_dt'] = marketing_events['finish_dt'].astype('datetime64')
ab_events['event_dt'] = ab_events['event_dt'].astype('datetime64')
ab_new_user['first_date'] = ab_new_user['first_date'].astype('datetime64')

print(marketing_events.info())
print(ab_events.info())
print(ab_new_user.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   name       14 non-null     object        
 1   regions    14 non-null     object        
 2   start_dt   14 non-null     datetime64[ns]
 3   finish_dt  14 non-null     datetime64[ns]
dtypes: datetime64[ns](2), object(2)
memory usage: 576.0+ bytes
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     62740 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61733 entries, 0

<a id='prepare_str'></a>
## Функция обработки строковых значений ##

In [11]:
def prepare_str(frame_name):
    for column in frame_name.columns:
        if frame_name.dtypes[column] == 'object':
            frame_name[column] = frame_name[column].str.strip().str.lower()

<a id='prepare_lower'></a>
## Перевод строковых значений в нижний регистр ##

In [12]:
prepare_str(frame_name=marketing_events)
prepare_str(frame_name=ab_events)
prepare_str(frame_name=ab_new_user)
prepare_str(frame_name=ab_participants)

<a id='prepare_nan'></a>
## Работа с пропусками ##

In [14]:
ab_events['details'] = ab_events['details'].fillna(0)
ab_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440317 entries, 0 to 440316
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   user_id     440317 non-null  object        
 1   event_dt    440317 non-null  datetime64[ns]
 2   event_name  440317 non-null  object        
 3   details     440317 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.4+ MB


<a id='prepare_dubl'></a>
## Работа с дубликатами ##

In [17]:
print(marketing_events.duplicated().sum())
print(ab_events.duplicated().sum())
print(ab_new_user.duplicated().sum())
print(ab_participants.duplicated().sum())

0
0
0
0


## Итоги предобработки данных ##
1. приведены к типу datetime:
    * marketing_events['start_dt']
    * marketing_events['finish_dt']
    * ab_events['event_dt']
    * ab_new_user['first_date']

2. строковые значения переведены в нижний регистр
3. пропуски ab_events['details'] заменены на 0
4. дубликаты отсутствуют